### Detrital Zircon U-Pb Table ###
This notebook contains code used to create the supplementary table containing all original detrital zircon U-Pb data.

In [ ]:
# Imports
import os
import pandas as pd
import numpy as np

In [ ]:
# Identify directory for U-Pb data
directory = 'upb_data'

In [ ]:
# Make a class for raw DZ data
class RawDZ:
    def __init__(self,name,accepted,rejected,standards,syst_238,syst_207):

        self.name = name
        self.accepted = accepted
        self.rejected = rejected
        self.standards = standards
        self.syst_238 = syst_238
        self.syst_207 = syst_207

        return

In [ ]:
# Get individual data tables and clean
samples = []

for root,dirs,files in os.walk(directory):
    # Get names from dirs
    if len(dirs)>1:
        names = dirs
        print(names)
    
    for file in files:
        if ('_DataTable.xls' in file) & ('lock' not in file):
            path = os.path.join(root,file)
            
            # Read relevant part of the Excel file
            df = pd.read_excel(path,skiprows=25,usecols='A:T',header=None,sheet_name='Sheet1')
            df.columns = np.arange(20)

            reject = pd.read_excel(path,skiprows=25,usecols='W:AP',header=None,sheet_name='Sheet1')
            reject.columns = np.arange(20)

            stand = pd.read_excel(path,skiprows=25,usecols='AS:BL',header=None,sheet_name='Sheet1')
            stand.columns = np.arange(20)

            syst = pd.read_excel(path,skiprows=9,nrows=2,header=None,
                                        usecols='B')

            syst_238 = syst.loc[0,1]
            syst_207 = syst.loc[1,1]
            
            # Remove empty rows
            df.dropna(how='all',inplace=True)
            reject.dropna(how='all',inplace=True)
            
            # Move SLs to standards
            sl_bool = df.iloc[:,0].str.contains('SL')

            sl = df[sl_bool==True]
            df = df[sl_bool==False]

            # Remove rejected standards
            reject.iloc[:,0] = reject.iloc[:,0].astype(str)

            std_reject = (
                (reject.iloc[:,0].str.contains('SL'))|
                (reject.iloc[:,0].str.contains('F'))|
                (reject.iloc[:,0].str.contains('R'))
            )

            reject = reject[std_reject==False]

            name = file.partition('_')[0]

            sample = RawDZ(name=name,accepted=df,rejected=reject,standards=stand,syst_238=syst_238,syst_207=syst_207)
            print(sample.name)

            samples.append(sample)

In [ ]:
# Set up the table

title = 'Table S1: Results of U-Th-Pb analyses for detrital zircon samples'
ncols = 20

title_row = pd.Series(index=np.arange(20),dtype='str')
title_row[0] = title
title_row[1:] = ''

header1 = pd.Series(index=np.arange(20),dtype='str')
header1[6] = 'Isotope ratios'
header1[11] = 'Apparent ages (Ma)'
header1[:6] = ''
header1[12:] = ''

header2_vals = (
['Analysis','U','206Pb','U/Th','206Pb*','±','207Pb*','±','206Pb*','±','error','206Pb*','±','207Pb*','±','206Pb*','±','Best age','±','Conc']
)
header2 = pd.Series(header2_vals)

header3_vals = (
    ['','(ppm)','204Pb','','207Pb*','(%)','235U*','(%)','238U','(%)','corr.','238U*','(Ma)','235U','(Ma)','207Pb*','(Ma)','(Ma)','(Ma)','(%)']
)

header3 = pd.Series(header3_vals)

empty_series = pd.Series(index=np.arange(20),dtype='str')
empty_series[:] = ''

reject_row = pd.Series(index=np.arange(20),dtype='str')
reject_row[0] = 'Rejected Analyses'

In [ ]:
# Create large Pandas dataframe

df = pd.DataFrame(columns = np.arange(20))
df.loc[0,:] = title_row
df.loc[1,:] = empty_series
df.loc[2,:] = header1
df.loc[3,:] = empty_series
df.loc[4,:] = header2
df.loc[5,:] = header3
df.loc[6,:] = empty_series

nickname_dic = {'CT15076':'KZ1','CT15082':'KZ2','CT15092':'KZ3','CT15127':'KZ4','CT15099':'KZ5','CT15113':'KZ6','AB0913':'RU1','CT130918-2A':'SV1',
'100211-9A':'SV2','CT130918-9A':'SV3','CT130919-5A':'SV4','100411-5':'SV5','CT130919-8A':'SV6','AB0926':'SV7','100211-1A':'SV8'}

for sample in samples:
    sample.nickname = nickname_dic[sample.name]
    
    nick_row = pd.Series(index=np.arange(20),dtype='str')
    nick_row[0] = sample.nickname + ' (' + sample.name + ')'
    nick_row[1:] = ''

    df.loc[df.shape[0],:] = nick_row

    df = pd.concat([df,sample.accepted],axis=0,ignore_index=True)

    df.loc[df.shape[0],:] = empty_series
    df.loc[df.shape[0],:] = reject_row

    df = pd.concat([df,sample.rejected],axis=0,ignore_index=True)

    df.loc[df.shape[0],:] = empty_series

print(df)

# Save the dataframe to a CSV
df.to_csv('table.csv',encoding='utf-8-sig',header=False,index=False)

In [ ]:
# Output data for R processing in IsoplotR

os.makedirs('isoplot',exist_ok=True)

for sample in samples:
    full_table = sample.accepted
    new_columns = (
['Analysis','U','206Pb/204Pb','U/Th','206Pb/207Pb','206Pb/207Pb_err','207Pb/235U','207Pb/235U_err','206Pb/238U','206Pb/238U_err','error_corr','206Pb/238U_age','206Pb/238U_ageerr','207Pb/235U_age','207Pb/235U_ageerr','206Pb/207Pb_age','206Pb/207Pb_ageerr','Best age','Best age_err','Conc']
)
    full_table.columns = new_columns

    # Set up new df in IsoplotR format (flipped ratios and 1se abs)
    working_table = pd.DataFrame([],columns=['U238Pb206','errU238Pb206','Pb207Pb206','errPb207Pb206'])
   
    # Flip ratios
    working_table['U238Pb206'] = 1/full_table['206Pb/238U']
    working_table['Pb207Pb206'] = 1/full_table['206Pb/207Pb']

    # Convert 2s percentage errors to 1s absolute errors
    working_table['errU238Pb206'] =  working_table['U238Pb206']*(full_table['206Pb/238U_err']/100)/2
    working_table['errPb207Pb206'] = working_table['Pb207Pb206']*(full_table['206Pb/207Pb_err']/100)/2

    #filename = sample.nickname + '_' + sample.name + '.csv'

    working_table.to_csv('isoplot/'+sample.nickname+'.csv',index=None)